In [14]:
import azureml.core
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

In [15]:
datastore = Datastore.get(ws,'mlteststore')

In [16]:
from azureml.core.compute import ComputeTarget , AmlCompute
compute_name = 'nallaballeramana1'
vm_size = 'Standard_E4as_v4'
compute_target = ws.compute_targets[compute_name]

In [17]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment

aml_run_config = RunConfiguration()
#compute target as defined in Azure machine learning compute section above
aml_run_config.target = compute_target

USE_CURATED_ENV = True

if USE_CURATED_ENV:
    curated_environment = Environment.get(workspace = ws, name = 'AzureML-sklearn-0.24-ubuntu18.04-py37-cpu')
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies =False

    #add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies =CondaDependencies.create(
        conda_packages = ['Pandas','scikit-learn'],
        pip_packages = ['azureml-sdk', 'azureml-dataset-runtime[fuse,pandas]'],
        pin_sdk_version = False
    )


In [26]:
from azureml.pipeline.steps import PythonScriptStep
#dataprep_source_dir = "./dataprep_src"

entry_point = 'Python_Script1.py'

In [27]:
py_script_run_step = PythonScriptStep(
    script_name = entry_point,
    compute_target = compute_target,
    arguments = ['--input-data', 'train.csv'], ## if we need to pass dataset as input to python script from pipeline
    runconfig = aml_run_config,
    allow_reuse = False
)

In [28]:
pipeline_steps = [py_script_run_step]

In [29]:
from azureml.pipeline.core import Pipeline
#building the pipelne
pipeline1 = Pipeline(workspace = ws, steps= [pipeline_steps])

In [30]:
from azureml.core import Experiment
#submit the pipeline to be run
pipeline_run1 = Experiment(ws, 'Second_trial_of_custom_pipeline').submit(pipeline1)
pipeline_run1.wait_for_completion(show_output = True)

Created step Python_Script1.py [3a079058][d26dbe42-2925-4789-8233-fd5fe4dc100d], (This step will run and generate new outputs)
Submitted PipelineRun 42709df4-b642-494e-9063-bd7a95ff2834
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/42709df4-b642-494e-9063-bd7a95ff2834?wsid=/subscriptions/3739307c-4f3c-4bc6-b980-08d7dccd9bbd/resourcegroups/testgroup/workspaces/ml_testing_space&tid=45365ad3-eb1f-4a19-9433-e9528c41f196
PipelineRunId: 42709df4-b642-494e-9063-bd7a95ff2834
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/42709df4-b642-494e-9063-bd7a95ff2834?wsid=/subscriptions/3739307c-4f3c-4bc6-b980-08d7dccd9bbd/resourcegroups/testgroup/workspaces/ml_testing_space&tid=45365ad3-eb1f-4a19-9433-e9528c41f196
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 4def989b-7bc6-4898-8f48-21b957dcef85
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/4def989b-7bc6-4898-8f48-21b957dcef85?wsid=/subscriptions/3739307c-4f3c-4bc6-b980-0

'Finished'